This Python script is used to process a DataFrame from a CSV file named 'train.csv'. It does the following functionality:
1. Creates a new column is_center, which is used to identify the correct offset for each EEG

The end result is a DataFrame where each sequence of rows with the same 'concatenated_scores' and 'eeg_id' has one row (the middle one, if the sequence length is odd, or the first one of the middle two, if the sequence length is even) marked as 'is_center'.

In [48]:
import pandas as pd

df = pd.read_csv('train.csv')
# Concat the last 6 columns into one
df['concatenated_scores'] = df.iloc[:, -6:].apply(lambda x: ''.join(x.astype(str)), axis=1)
# Create column of False
df['is_center'] = False

# Sort by eeg_id and then eeg_sub_id
df = df.sort_values(['eeg_id', 'eeg_sub_id'])

# Iterate over the rows
counter = 0
start_row_index = 0
for i in range(len(df)):
    if i == 0:
        counter += 1
        continue
    
    # Check if the concatenated_scores is the same as the previous row
    if df.at[i, 'concatenated_scores'] == df.at[i-1, 'concatenated_scores'] and df.at[i, 'eeg_id'] == df.at[i-1, 'eeg_id']:
        counter += 1
    else:
        row_to_edit = 0
        if counter == 1:
            row_to_edit = start_row_index
        else:
            row_to_edit = int((counter - 1) / 2) + start_row_index
        df.at[row_to_edit, 'is_center'] = True
        start_row_index = i
        counter = 1


row_to_edit = int((counter - 1) / 2) + start_row_index
df.at[row_to_edit, 'is_center'] = True

In [49]:
# Create a column that is the count of the number of True's that appear in is_center for each combination of eeg_id and concatenated_scores
df['center_count'] = df.groupby(['eeg_id', 'concatenated_scores'])['is_center'].transform('sum')

In [50]:
# Print value counts for center_count
print(df['center_count'].value_counts())

center_count
1     98365
2      6453
3       693
4       269
5       249
6       130
13      110
18       85
7        78
11       66
30       66
29       54
24       52
27       34
16       32
21       30
15       17
8        17
Name: count, dtype: int64


In [51]:
# Filter where center_count is 11
df[df['center_count'] == 0]

,eeg_id,eeg_sub_id,eeg_label_offset_seconds,spectrogram_id,spectrogram_sub_id,spectrogram_label_offset_seconds,label_id,patient_id,expert_consensus,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote,concatenated_scores,is_center,center_count


In [47]:
print(len(df))

106800
